In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing

In [ ]:
df=pd.read_csv("dataset_sdn.csv")
df

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
column_names= df.columns
column_names

In [ ]:
df.isnull().sum().plot.bar()
plt.title("NULL Values for each column ")
plt.xlabel("Column names")
plt.ylabel("Count")

In [ ]:
df=df.dropna()

In [ ]:
df.info()

In [ ]:
y=df['label']
x=df.drop(['label'],axis=1)
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=104,test_size=0.2)
train_data=pd.concat([x_train,y_train],axis=1,join='inner')

correl=train_data.corr(numeric_only=True)['label'].sort_values()
correl.drop('label',inplace=True)

In [ ]:
selected=correl.loc[abs(correl)>0.08]
attr_selected=list(selected.index)
print(attr_selected)


In [ ]:
norm_x_train_data=(x_train.select_dtypes(include=['float64','int64'])-x_train.min(numeric_only=True))
norm_x_test_data=(x_test.select_dtypes(include=['float64','int64'])-x_test.min(numeric_only=True))
norm_x_train_data=preprocessing.scale(norm_x_train_data[attr_selected])
norm_x_test_data=preprocessing.scale(norm_x_test_data[attr_selected])

norm_x_train_data.shape
y_train.shape

In [ ]:
train_x,val_x,train_y,val_y=train_test_split(norm_x_train_data,y_train,stratify=y_train,test_size=0.2,random_state=0)

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb=GaussianNB()
nb.fit(norm_x_train_data,y_train)
nb.score(norm_x_test_data,y_test)


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier()

knn.fit(norm_x_train_data,y_train)
knn.score(norm_x_test_data,y_test)

In [ ]:
from sklearn.svm import SVC
svc=SVC()

svc.fit(norm_x_train_data,y_train)
svc.score(norm_x_test_data,y_test)

HYBRID MODEL TESTING


In [ ]:
train_x,val_x,train_y,val_y=train_test_split(norm_x_train_data,y_train,stratify=y_train,test_size=0.3,random_state=0)

In [ ]:
print(norm_x_train_data.shape,norm_x_test_data.shape)

In [ ]:
model1=SVC(kernel='rbf',gamma='auto')
model1.fit(train_x,train_y)
svc_val_pred=model1.predict(val_x)
svc_val_pred=pd.DataFrame(svc_val_pred)

svc_test_pred=model1.predict(norm_x_test_data)
svc_test_pred=pd.DataFrame(svc_val_pred)

In [ ]:
model2=KNeighborsClassifier(n_neighbors=5)
model2.fit(train_x,train_y)
knn_val_pred=model1.predict(val_x)
knn_val_pred=pd.DataFrame(knn_val_pred)

knn_test_pred=model1.predict(norm_x_test_data)
knn_test_pred=pd.DataFrame(knn_val_pred)

In [ ]:
model3=GaussianNB()
model3.fit(train_x,train_y)
nb_val_pred=model1.predict(val_x)
nb_val_pred=pd.DataFrame(nb_val_pred)

nb_test_pred=model1.predict(norm_x_test_data)
nb_test_pred=pd.DataFrame(nb_val_pred)

In [ ]:
val_input=pd.concat([pd.DataFrame(val_x,columns=column_names),knn_val_pred,svc_val_pred,nb_val_pred],axis=1)
test_input=pd.concat([pd.DataFrame(x_test,column_names),knn_test_pred,svc_test_pred,nb_test_pred],axis=1)

In [ ]:
model=RandomForestClassifier()
model.fit(val_input,val_y)
# y_pred=model.predict(x_test)
print(model.score(test_input,y_test))
# print(metrics.accuracy_score(test_input,y_pred))